<h1 style = "font-size: 30px; text-align: center;">AI Genetic Hands On</h1>
<h2 style = "font-size: 25px; text-align: center;">Hospital Job Scheduling</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Name: Ali Bahari</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Student Id: 810196688</h2>
<h4 style="text-align: center">Spring 1400</h4>

<h2 style = "font-size: 25px;">Notes</h2>

<h3>Tests: </h3>

<p style="text-indent :2em;"><b>Test1: </b><mark>Average time < 15s</mark></p>
<p style="text-indent :2em;"><b>Test2: </b><mark>Average time < 60s</mark></p>
    

<h3>Outputs: </h3>
<p style="text-indent :2em;">Respectively <mark>output1.txt</mark> and <mark>output2.txt</mark></p>

<br>

Project Name: Hands-on 1

Project Goal: Using genetic algorithm to assign doctors in a hospital to different shifts in a day (3 shifts: morning, evening and night) according to some constraints and rules.

The constraints:
1. There are min and max number of doctors that can be assigned to a shift.
2. A doctor that has worked the night shift can not work morning and evening shift the next day.
3. A doctor can not work there night shifts a row.
4. Every doctor must work a specific number of shifts.

All these constraints are given as input, now we have to incorporate genetic algorithms to assign doctors to the shifts.

Everything has been implemented, tested and run on Linux but the report has been done on Windows!

<h2 style = "font-size: 25px;">Test Files And Imports</h2>

In [23]:
import time
import random
import bisect
import os.path

testFile1 = "test1.txt"
testFile2 = "test2.txt"

<h2 style = "font-size: 25px;">Reading Test File Content</h2>

In [24]:
def readInput(testFile) :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    
    allShifts = []
    for i in range(2, days + 2):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))

    file.close()
    return [days, list(range(doctors)), maxCapacity, allShifts]

Test cases are given. In each test case the constraints of the problem is given so we will use them in our genetic algorithm implementation.

<h2 style = "font-size: 25px;">Job Scheduler</h2>

In [25]:
class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.allShifts = fileInfo[3]
        self.popSize = 350
        # [[[]] , [[] ==> gene(shift)] ==> chromosome(one assignment)] ==> chromosomes(all assignments)
        self.chromosomes = self.generateInitialPopulation()
        self.elitismPercentage = 0.12
        self.pc = 0.8
        self.pm = 0.15
        self.bestFitness = float("inf")
        self.repetitaveFitnesses = 0
        self.rounds = 0
        self.numGeneration = 1
        
    def createOneGene(self):
        num_docs = random.randint(0 , self.doctors)
        visited_docs = [False] * self.doctors
        gene = []
        
        for _ in range(num_docs):
            doc_ID = random.randint(0 , self.doctors - 1)
            while(visited_docs[doc_ID]):
                doc_ID = random.randint(0 , self.doctors - 1)
            
            gene.append(doc_ID)
            visited_docs[doc_ID] = True
            
        return gene
    
    
    def generateInitialPopulation(self):
        chromosomes_list = []
        for _ in range(self.popSize):
            chromosome = []
            for _ in range(self.days * 3):
                chromosome.append(self.createOneGene())
            chromosomes_list.append(chromosome)
            
        return chromosomes_list;
        
    
    def crossOver(self, chromosome1, chromosome2):
        if(random.random() <= self.pc):
            # # One-point
            # slice_point = random.randint(0 , len(chromosome1) - 1)
            # return (chromosome1[0 : slice_point] + chromosome2[slice_point:] ,
            #         chromosome2[0 : slice_point] + chromosome1[slice_point:])


            # # Two-point
            # slice_point_1 = random.randint(0 , len(chromosome1) - 1)
            # slice_point_2 = random.randint(slice_point_1 , len(chromosome1) - 1)
            # return (chromosome1[0 : slice_point_1] + chromosome2[slice_point_1 : slice_point_2] + chromosome1[slice_point_2:] ,
            #         chromosome2[0 : slice_point_1] + chromosome1[slice_point_1 : slice_point_2] + chromosome2[slice_point_2:])

            # Three-point
            slice_point_1 = random.randint(0 , len(chromosome1) - 1)
            slice_point_2 = random.randint(slice_point_1 , len(chromosome1) - 1)
            slice_point_3 = random.randint(slice_point_2 , len(chromosome1) - 1)
            return (chromosome1[0 : slice_point_1] + chromosome2[slice_point_1 : slice_point_2] + chromosome1[slice_point_2 : slice_point_3] + chromosome2[slice_point_3:] ,
                    chromosome2[0 : slice_point_1] + chromosome1[slice_point_1 : slice_point_2] + chromosome2[slice_point_2 : slice_point_3] + chromosome1[slice_point_3:])            
        
        return chromosome1 , chromosome2
    
    
    # def mutate_gene(self , gene):
    #     possible_docs = []

    #     for docID in self.doctorsIds:
    #         if(docID not in gene):
    #             possible_docs.append(docID)
        

    #     has_been_added_docs = [False] * len(possible_docs)
        
    #     num_toBeMutated = random.randint(0 , len(possible_docs))

    #     new_gene = []
    #     for _ in range(num_toBeMutated):
    #         index_added = random.randint(0 , len(possible_docs) - 1)
    #         while(has_been_added_docs[index_added]):
    #             index_added = random.randint(0 , len(possible_docs) - 1)
    #         has_been_added_docs[index_added] = True
            
    #         new_gene.append(possible_docs[index_added])  
  
            
    #     if(num_toBeMutated == 0):
    #         gene = self.createOneGene()
        
    #     return gene
    
                
    # def mutate(self, chromosome):
    #     if(random.random() <= self.pm):
    #         num_toBeMutated = random.randint(len(chromosome) // 2 , len(chromosome) - 1)  # change at least 50% of shifts
    #         visited_shifts = [False] * len(chromosome)
    #         for i in range(num_toBeMutated):
    #             index_toBeMutated = random.randint(0 , len(chromosome) - 1)
    #             while(visited_shifts[index_toBeMutated]):
    #                 index_toBeMutated = random.randint(0 , len(chromosome) - 1)            
    #             visited_shifts[index_toBeMutated] = True
                
    #             chromosome[index_toBeMutated] = self.mutate_gene(chromosome[index_toBeMutated])
             
    #     return chromosome

    def mutate(self, chromosome):
        if(random.random() <= self.pm):
            num_CoupletoBeMutated = random.randint(1 , len(chromosome) // 2)  # change at least 50% of shifts
            visited_shifts = [False] * len(chromosome)
            for i in range(num_CoupletoBeMutated):
                index_toBeMutated_1 = random.randint(0 , len(chromosome) - 1)
                while(visited_shifts[index_toBeMutated_1]):
                    index_toBeMutated_1 = random.randint(0 , len(chromosome) - 1)          
                visited_shifts[index_toBeMutated_1] = True

                index_toBeMutated_2 = random.randint(0 , len(chromosome) - 1)
                while(visited_shifts[index_toBeMutated_2]):
                    index_toBeMutated_2 = random.randint(0 , len(chromosome) - 1)          
                visited_shifts[index_toBeMutated_2] = True                
                
                temp = chromosome[index_toBeMutated_1]
                chromosome[index_toBeMutated_1] = chromosome[index_toBeMutated_2]
                chromosome[index_toBeMutated_2] = temp
             
        return chromosome
        
        
    def checkShiftConstraints(self , chromosome , nightIndex):
        num_conflicts = 0
        for doc in chromosome[nightIndex]:
            if(doc in chromosome[nightIndex + 1]): # Tomorrow morning
                num_conflicts += 1
            
            if(doc in chromosome[nightIndex + 2]): # Tomorrow evening
                num_conflicts += 1
                
            if(doc in chromosome[nightIndex + 3]): # Three night shifts in a row
                if(nightIndex + 6 < len(chromosome) and doc in chromosome[nightIndex + 6]):
                    num_conflicts += 1

        return num_conflicts
    

    def checkCapacityAndRangeConstraints(self , chromosome , doc_id , fitness):
        num_shifts = 0
        for i in range(len(chromosome)):
            shift = chromosome[i]
            
            if(doc_id in shift):
                num_shifts += 1
            
            # Range Constraints
            if(doc_id == 0):
                if(len(shift) < self.allShifts[i // 3][i % 3][0]):
                    fitness += (self.allShifts[i // 3][i % 3][0] - len(shift))
                elif(len(shift) > self.allShifts[i // 3][i % 3][1]):
                    fitness += (len(shift) - self.allShifts[i // 3][i % 3][1])
                

        if(num_shifts > self.maxCapacity):
            fitness += 1

        return fitness
                
                
    def calculateFitness(self, chromosome):
        fitness = 0
        for nightIndex in range(2 , len(chromosome) - 1 , 3):
            fitness += self.checkShiftConstraints(chromosome , nightIndex)
                
        for doc_id in self.doctorsIds:
            fitness = self.checkCapacityAndRangeConstraints(chromosome , doc_id , fitness)
        
        
        return fitness
    
    
    def getTwoParentsForCrossOver(self , chromosomes , fitnesses , max_fitness):
        probability = []
        for i in range(len(fitnesses)):
            probability += ([i] * (max_fitness - fitnesses[i] + 1))
        
        idx_1 = random.randint(0 , len(probability) - 1)
        idx_2 = random.randint(0 , len(probability) - 1)
        while(idx_1 == idx_2):
            idx_2 = random.randint(0 , len(probability) - 1)

        return chromosomes[probability[idx_1]] , chromosomes[probability[idx_2]]
    
    
    def generateNewPopulation(self):
        new_generation = []
        sortedChromosomes = []
        fitnesses = [];
        for i in range(self.popSize):
            chromosome = self.chromosomes[i]
            current_fitness = self.calculateFitness(chromosome)
            idx = bisect.bisect(fitnesses , current_fitness)
            fitnesses.insert(idx , current_fitness)
            sortedChromosomes.insert(idx , chromosome)
        
        fitnesses = fitnesses[::-1]
        sortedChromosomes = sortedChromosomes[::-1]
        max_fitness = fitnesses[0]

        # elitism        
        for _ in range(int(self.elitismPercentage * self.popSize)):
            new_generation.append(sortedChromosomes.pop())
            fitnesses.pop()

        
        # Crossover
        for _ in range(0 , len(sortedChromosomes) , 2):
            parent_1 , parent_2 = self.getTwoParentsForCrossOver(sortedChromosomes , fitnesses , max_fitness);
            child_1 , child_2 = self.crossOver(parent_1 , parent_2)
            new_generation.append(self.mutate(child_1))
            new_generation.append(self.mutate(child_2))
            
        return new_generation
    
    def hasReachedGoal(self , printBestFitnessForEachGeneration = False):
        prevBestFitness = self.bestFitness
        for chromosome in self.chromosomes:
            current_fitness = self.calculateFitness(chromosome)
            if(current_fitness < self.bestFitness):
                self.bestFitness = current_fitness
                
            if(current_fitness == 0):
                return chromosome

        if(printBestFitnessForEachGeneration):
            print("Best fitness of generation " + str(self.numGeneration) + ": " + str(self.bestFitness))
            self.numGeneration += 1
        
        if(prevBestFitness == self.bestFitness):
            self.repetitaveFitnesses += 1
        else:
            self.repetitaveFitnesses = 0

        if(self.repetitaveFitnesses == 60 and self.rounds % 2 == 0):
            self.rounds += 1
            self.pc = 1
            self.pm = 0
            self.repetitaveFitnesses = 0
        elif(self.repetitaveFitnesses == 60 and self.rounds % 2 == 1):
            self.rounds += 1
            self.pc = 0.8
            self.pm = 0.15
            self.repetitaveFitnesses = 0

        return None
    
    def schedule(self):
        while(True):
            final_chromosome = self.hasReachedGoal()
            if(final_chromosome is not None):
                return final_chromosome
            
            self.chromosomes = self.generateNewPopulation()

# Implementation:

## Fitness:

For fitness the number of conflicts are considered meaning that for doctors' capacity, consecutive night shifts and night-morning-evening relation for each conflict we'll increment fitness by one. But for number of doctors in each shift the difference between the current number of doctors and the nearest range limit is considered. For example if current number of doctors is 5 and the range limits are (1,3), 5-3=2 will be added to fitness.

By this definition the higher the fitness value is, the worse the parent gets. There won't be any concerns since this issue is handled in chosing two parents.

## Crossover:

For crossover 3-point crossover is used meaning there'll be there point chosen randomly and then two chromosomes will be intertwined. 1-point and 2-point crossover are also implemented but since 3-point crossover has even more randomness to it, 3-point is used. Certainly the probability pc will have an importent role here.

## Mutation:

First Method:
For mutation random number of genes are selected and swapped, meaning a random gene will be chosen, then another random and different gene will be chosen, after that these two genes will be swapped. It's importent to note that every pair of genes chosen for swapping is unique meaning that every gene can be swapped with another gene only once. This method is faster than the following method.

Second Method (commented in code):
For mutation random number of genes are selected and mutated. For mutating a gene, random number of doctors that are not currently in this gene are selected and returned but if this random number is eqaul to zero then a new gene will be returned.
Certainly the probability pm will have an importent role here. This method sometimes exceeds time limit and also is a bit slower than the fisrt method so the first method was chosen.

## Initial Population:

For generating the initial population doctors are assigned to shifts randomly and also total number of each chromosome will be number of days multiplied by 3 since there are 3 shifts in each day.

## Choosing Parents:

For generating childs, parents are chosen based on their fitness value. Here first the max fitness is found and based on the explanation above this is the worst possible chromosome. Now, from each of the fitness values this max value will be subtracted this will change our concept of fitness. With these new fitness values each chromosome will get part of a list that acts like a roulette wheel. After all of these operations we'll choose a two random indices of this list which will finally return two chromosomes with regards to its fitness. The higher the fitness value, The higher chance it'll be selected. We have found the desired parents.

## Generating New Population:

With all the above functions implemented we can generate the new population. First we need all of the fitness values in sorted order so we could know and rank chromosomes. After passing some of the chromosomes based on elitism probability, the remaining chromosomes will be chosen to be crossed over or mutated based on pc and pm value. Finally, the new generation will be generated.

## Reaching The Goal:

New population will keep on generating until a generation reaches the goal which is zero fitness value.

For not getting stuck at a local minimum, changing pm and pc after being stuck is used. This means if after 60 generations the best fitness hasn't been changed then there will be new pm and pc values so that it would increase diversity and get us out of local minimum. Also if being stuck happens again pm and pc will be restored to their initial value. This cycle will keep on happening until we reach the goal.

## CREATING OUTPUT FILE

In [26]:
def generate_solution(assignment , testNumber):
    if(os.path.isfile("output" + str(testNumber) + ".txt")):
        open("output" + str(testNumber) + ".txt", 'w').close()
    
    for start_idx_day in range(0 , len(assignment) , 3):
        dayResult = ""
        for i in range(0 , 3):
            if(len(assignment[start_idx_day + i])) == 0:
                dayResult += "empty"
            else:
                dayResult += ",".join(map(str , assignment[start_idx_day + i]))
            
            if(i == 2):
                dayResult += "\n"
            else:
                dayResult += " "
            
        with open("output" + str(testNumber) + ".txt", "a") as outputFile:
            outputFile.write(dayResult)

generate_solution will convert the result to the desired format and write it to output1.txt or output2.txt

<h2 style = "font-size: 25px;">Execution</h2>

In [27]:
fileInfo1 = readInput(testFile1)

start = time.time()

scheduler = JobScheduler(fileInfo1)
result_assignment_test_1 = scheduler.schedule()

end = time.time()

print("test 1: ", '%.2f'%(end - start), 'sec')

generate_solution(result_assignment_test_1 , 1);

test 1:  12.91 sec


In [28]:
fileInfo2 = readInput(testFile2)

start = time.time()

scheduler = JobScheduler(fileInfo2)
result_assignment_test_2 = scheduler.schedule()

end = time.time()

print("test 2: ", '%.2f'%(end - start), 'sec')

generate_solution(result_assignment_test_2 , 2);

test 2:  20.06 sec


## Average Execution Time

Time was recorded on linux

### Test 1: 9.28 s

### Test 2 : 21.645 s

#### Notes

For executing the codes it's better to run code files provided in the project zip file.

bisect function is used from bisect library.